In [413]:
#需要导入的库或包
import xlrd
import pandas as pd
import numpy as np
from pandas import DataFrame
from scipy.interpolate import lagrange#拉格朗日函数

In [414]:
#读取文件
file_path = "D:\TTF\Course\数据挖掘\data-数据预处理数据-Students.xls"
data = pd.read_excel(file_path)

In [415]:
#“家庭人均年收入”变量值为空或错误值999999；
for i in range(len(data)):
    if data.iloc[i][2] > 100000:
        data = data.replace(data.iloc[i][2], data['家庭人均年收入'].mean())

In [416]:

#自定义列向量插值函数
def ploy(s,n,k=6):
    y=s[list(range(n-k,n))+list(range(n+1,n+1+k))]#取数
    y=y[y.notnull()]
    return lagrange(y.index,list(y))(n)
for i in data.columns:
    for j in range(len(data)):
        if(data[i].isnull())[j]:
            data[i][j] = ploy(data[i],j)
#data
#data.to_excel('D:\TTF\data.xls')

D:\Anaconda3\lib\site-packages\pandas\core\series.py:841: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [417]:
#计算变量“在校综合评价指数”，标准化  （在校综合评价指数-61）/（138-61）*100
data.在校综合评价指数 = (data.在校综合评价指数-61)/(138-61)*100
#data

In [418]:
#将变量“是否无偿献血”中的0,1改写成yes，no
K = data.是否无偿献血
K[K == '1'] = 'Yes'
K[K == '0'] = 'No'
#data

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [419]:

#对“家庭年收入”变量做离散化处理
K = data.家庭人均年收入
#pd.cut(data.家庭人均年收入,3)

In [420]:
#随机分割生成样本集分割变量
#将样本随机的划分成3个集合，训练样本集（Training）40、检验样本集（Testing）20、验证样本集（Validation）20
S1 = data.sample(n=40)
S2 = data.sample(n=20)
S3 = data.sample(n=20)
#print(S1)
#print(S2)
#print(S3)

In [421]:
#读取两张表中的老生和新生的数据并合并
DATA_DIR = 'D:/TTF/Course/数据挖掘/' 
excel_name = '%sdata-数据预处理数据-Students.xls' % DATA_DIR
wb = xlrd.open_workbook(excel_name)
 
# 获取workbook中所有的表格
sheets = wb.sheet_names()
 
# 循环遍历所有sheet
df_28 = DataFrame()
for i in range(len(sheets)):
    # skiprows=2 忽略前两行
    df = pd.read_excel(excel_name,sheet_name=i,index=False, encoding='utf8')
    df_28 = df_28.append(df)
#df_28

In [393]:
R_file_path = "D:\TTF\Course\数据挖掘\data-数据预处理数据-ReportCard.xls"
data = pd.read_excel(R_file_path)

In [394]:
#计算每个学生的总成绩 
data['总分'] = data["思想品德"] + data["语文"] + data["英语"] + data['物理'] + data[ '化学'] + data['地理'] + data['历史']
#data

In [395]:
#对每个学生的每门成绩进行两级评定计算，大于等于60分，则评定为合格，否则评定为不合格。
for i in ["思想品德","语文","英语",'物理','化学','地理','历史']:
    data.loc[(data[i] >= 60), i+str(1)] = '及格'
    data.loc[(data[i] <  60), i+str(1)] = '不及格'
#data

In [396]:
#对每个学生的每门成绩按A、B、C、D、E进行多级评定计算。
for i in ["思想品德","语文","英语",'物理','化学','地理','历史']:
    data.loc[(data[i] >= 90),i+str(1)] = 'A'
    data.loc[(data[i] >= 80)&(data[i] < 90), i+str(1)] = 'B'
    data.loc[(data[i] >= 70)&(data[i] < 80), i+str(1)] = 'C'
    data.loc[(data[i] >= 60)&(data[i] < 70), i+str(1)] = 'D'
    data.loc[(data[i] < 60), i+str(1)] = 'E'
#data

In [397]:
#根据思想品德课程的得分对每个学生的总成绩进行调整，如果思想品德评定等级为A，则总成绩上浮1% 
for i in range(len(data)):
    if data.iloc[i][11] == 'A':
        data = data.replace(data.iloc[i][10],data.iloc[i][10]*1.01)
#data

In [398]:
#inplace默认为False,不改变oly;若inplace=True,则改变原有数据
#分性别将总成绩按降序排序
F = data[data.性别 == 1]
M = data[data.性别 == 2]
F = F.sort_values(by=['总分'],ascending=False,inplace=False) 
M = M.sort_values(by=['总分'],ascending=False,inplace=False) 
#print(F)
#print(M)

In [399]:
#筛选出总成绩大于500分的所有男生（性别为1）样本
T = data[(data.总分 > 500)&(data.性别 == 1)]
#T

In [400]:
#随机抽取70&的样本
S3 = data.sample(int(len(data)*0.7))
#S3

In [401]:
#分别计算男女生总成绩的平均值和标准差。
#F --  男  M -- 女
#mean()为均值  std()为标准差
F = data[data.性别 == 1]
M = data[data.性别 == 2]
Fm = data['总分'].mean()
Fs = data['总分'].std()           #统计数值型各列标准差
Mm = data['总分'].mean()
Ms = data['总分'].std()  
F_M = [Fm,Fs,Mm,Ms]
#F_M

In [422]:
# -*- coding: utf-8 -*-
 
import openpyxl
import codecs
from openpyxl.utils import get_column_letter
 
def txt_to_xlsx(filename,outfile):
 
    fr = codecs.open(filename,'r')
    wb = openpyxl.Workbook()
    ws = wb.active
    ws = wb.create_sheet()
    ws.title = 'Sheet1'
    row = 0
    for line in fr:
        row +=1
        line = line.strip()
        line = line.split('\t')
        col = 0
        for j in range(len(line)):
            col +=1
            #print (line[j])
            ws.cell(column = col,row = row,value = line[j].format(get_column_letter(col)))
    wb.save(outfile)
#读取xlsx内容    
def read_xlsx(filename):
    #载入文件
    wb = openpyxl.load_workbook(filename)
    #获取Sheet1工作表
    ws = wb.get_sheet_by_name('Sheet1')
    #按行读取
    for row in ws.rows:
        for cell in row:
            print (cell.value)
    #按列读
    for col in ws.columns:
        for cell in col:
            print (cell.value)
 
if __name__=='__main__':
    inputfileTxt = 'D:\TTF\Course\数据挖掘\data-数据预处理数据-DRUG.txt'
    outfileExcel = 'D:\TTF\Course\数据挖掘\data-数据预处理数据-DRUG.xlsx'
    txt_to_xlsx(inputfileTxt,outfileExcel)
    read_xlsx(outfileExcel)


Age,Sex,BP,Cholesterol,Na,K,Drug
23,F,HIGH,HIGH,0.792535,0.031258,drugY
47,M,LOW,HIGH,0.739309,0.056468,drugC
47,M,LOW,HIGH,0.697269,0.068944,drugC
28,F,NORMAL,HIGH,0.563682,0.072289,drugX
61,F,LOW,HIGH,0.559294,0.030998,drugY
22,F,NORMAL,HIGH,0.676901,0.078647,drugX
49,F,NORMAL,HIGH,0.789637,0.048518,drugY
41,M,LOW,HIGH,0.766635,0.069461,drugC
60,M,NORMAL,HIGH,0.777205,0.05123,drugY
43,M,LOW,NORMAL,0.526102,0.027164,drugY
47,F,LOW,HIGH,0.896056,0.076147,drugC
34,F,HIGH,NORMAL,0.667775,0.034782,drugY
43,M,LOW,HIGH,0.626527,0.040746,drugY
74,F,LOW,HIGH,0.792674,0.037851,drugY
50,F,NORMAL,HIGH,0.82778,0.065166,drugX
16,F,HIGH,NORMAL,0.833837,0.053742,drugY
69,M,LOW,NORMAL,0.848948,0.074111,drugX
43,M,HIGH,HIGH,0.656371,0.046979,drugA
23,M,LOW,HIGH,0.55906,0.076609,drugC
32,F,HIGH,NORMAL,0.643455,0.024773,drugY
57,M,LOW,NORMAL,0.536746,0.028061,drugY
63,M,NORMAL,HIGH,0.616117,0.023773,drugY
47,M,LOW,NORMAL,0.809199,0.026472,drugY
48,F,LOW,HIGH,0.87444,0.058155,drugY
33,F,LOW,HIGH,0.858387

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).


In [426]:
# -*- encoding: utf-8 -*-
import xlwt #需要的模块
 
def txt_xls(filename,xlsname):
    """
    :文本转换成xls的函数
    :param filename txt文本文件名称、
    :param xlsname 表示转换后的excel文件名
    """
    try:
        f = open(filename) 
        xls=xlwt.Workbook()
        #生成excel的方法，声明excel
        sheet = xls.add_sheet('sheet1',cell_overwrite_ok=True) 
        x = 0 
        while True:
            #按行循环，读取文本文件
            line = f.readline() 
            if not line: 
                break  #如果没有内容，则退出循环
            for i in range(len(line.split('\t'))):
                item=line.split('\t')[i]
                sheet.write(x,i,item) #x单元格经度，i 单元格纬度
            x += 1 #excel另起一行
        f.close()
        xls.save(xlsname) #保存xls文件
    except:
        raise
if __name__ == "__main__" :
    filename =  'D:\TTF\Course\数据挖掘\data-数据预处理数据-DRUG.txt'
    xlsname  =  'D:\TTF\Course\数据挖掘\data-数据预处理数据-DRUG1.xls'
    txt_xls(filename,xlsname)


<input>:30: DeprecationWarning: invalid escape sequence \T
<input>:31: DeprecationWarning: invalid escape sequence \T
<input>:30: DeprecationWarning: invalid escape sequence \T
<input>:31: DeprecationWarning: invalid escape sequence \T
<input>:30: DeprecationWarning: invalid escape sequence \T
<input>:31: DeprecationWarning: invalid escape sequence \T
<ipython-input-426-1cb9dc0e743e>:30: DeprecationWarning: invalid escape sequence \T
  filename =  'D:\TTF\Course\数据挖掘\data-数据预处理数据-DRUG.txt'
<ipython-input-426-1cb9dc0e743e>:31: DeprecationWarning: invalid escape sequence \T
  xlsname  =  'D:\TTF\Course\数据挖掘\data-数据预处理数据-DRUG1.xls'
